# Using Google Directions API for speed calculation

You can generate and send google directions API requests based on the network. The process will create a 'car' modal
subgraph and generate API requests for all edges in the subgraph. The number of requests is at most the number of edges
in the subgraph. The process simplifies edges using `osmnx` library's method to extract a chains of nodes with no
intersections, in this way reducing the number of requests. If your graph is already simplified, the number of requests
will be equal to the number of edges.

In [1]:
# read sample network
from genet import Network
from genet import google_directions
import os

n = Network('epsg:27700')
path_to_matsim_network = '../example_data/pt2matsim_network'
n.read_matsim_network(os.path.join(path_to_matsim_network, 'network.xml'))
n.read_matsim_schedule(os.path.join(path_to_matsim_network, 'schedule.xml'))
n.print()

Graph info: Name: Network graph
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of unique routes: 68
Number of stops: 45


To send requests to Google Direction API you need a key [(read more here)](https://developers.google.com/maps/documentation/directions/start). After obtaining a key, you can either pass it to the elevant function directly

Or set it as an environmental variable called `GOOGLE_DIR_API_KEY`, if using command line: `$ export GOOGLE_DIR_API_KEY='key'`

If you use AWS, you can also store the key in the `Secrets Manager` [(read more here)](https://aws.amazon.com/secrets-manager/)
authenticate to your AWS account and then pass the `secret_name` and `region_name` to the `send_requests_for_network` 
method:

This method will save derived results in the output directory provided, an example can be found here: 
`../example_data/example_google_speed_data`. It comprises of the google polyline of the response and speed derived from
distance and time taken to travel as well as information that was generated in order to make the response such as
the node ids in the network for which this response holds, the `path_nodes` which denote any extra nodes from the non
simplified chain of nodes/edges in the request, the polyline of the network path, encoded using the same polyline 
encoding as the Google request polyline; as well as spatial information about the origin and destination of the request
and timestamp.

You can read saved results:

In [2]:
api_requests = google_directions.read_saved_api_results('../example_data/example_google_speed_data')

In [3]:
api_requests

{('9791490', '4698712638'): {'path_nodes': ('9791490', '4698712638'),
  'path_polyline': 'mvmyHpqYb@lA',
  'origin': {'id': '9791490',
   'x': 529414.5591563961,
   'y': 181898.4902840198,
   'lat': 51.5211862,
   'lon': -0.1360879,
   's2_id': 5221390682074967291},
  'destination': {'id': '4698712638',
   'x': 529387.9166476472,
   'y': 181877.74867097137,
   'lat': 51.5210059,
   'lon': -0.1364793,
   's2_id': 5221390682013665023},
  'timestamp': 1594385229.635254,
  'parsed_response': {'google_speed': 6.8, 'google_polyline': 'mvmyHpqYb@pA'}}}

Once you have results, you can match them to the network. This will create a dictionary of non-simplified edges to which the response data applies.

In [4]:
google_edge_data = google_directions.map_results_to_edges(api_requests)

In [5]:
google_edge_data

{('9791490', '4698712638'): {'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA'}}

In [6]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 4.166666666666667,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279}}

If we're working with a network that may have multiple edges between the same pair of nodes, we can restrict the links to which the data will be applied by specifying a modal condition, so that at least only links allowing cars will inherit this data.

In [7]:
def modal_condition(value):
    return 'car' in value

n.apply_attributes_to_edges(google_edge_data, conditions={'modes': modal_condition})

2020-12-17 13:05:09,582 - Changed Edge attributes for 1 edges


Resulting in two new data points in the relevant links: `google_speed` and `google_polyline`.

In [8]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 4.166666666666667,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279,
  'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA'}}

You can also choose to set google speed as the `freespeed` in the network. But be mindful if you use it for MATSim
simulations, `freespeed` denotes the maximum speed a vehicle can travel on a certain link, Google Directions API data
with `traffic=True` should be ran late at night/early morning ~4am local time to the network for any reliable results.
Otherwise you are adding traffic conditions to the network which should be simulated by demand (population) side of 
the model rather than supply (network).

In [9]:
def set_google_speed(link_attribs):
    return link_attribs['google_speed']

n.apply_function_to_links(set_google_speed, 'freespeed')

2020-12-17 13:05:09,671 - 3165 out of 3166 links have not been affected by the function. Links affected: ['596']
2020-12-17 13:05:09,679 - Changed Link attributes for 1 links


In [10]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 6.8,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279,
  'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA'}}